# Job Shop Scheduling問題をOR-Toolで解いてみる
https://developers.google.com/optimization/scheduling/job_shop?hl=ja

In [28]:
!pip install ortools


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
import collections
from ortools.sat.python import cp_model

### データの定義
Machine: 3  
Job: 5  
の条件で最適化を実施する

In [30]:
# Job情報
jobs_data = [  # task = (machine_id, processing_time).
    [(0, 3), (1, 2), (2, 2)],  # Job0
    [(0, 2), (2, 1), (1, 4)],  # Job1
    [(1, 4), (2, 3)],  # Job2
    [(1, 2), (0, 1), (2, 4)],  # Job3
    [(2, 1), (0, 2), (1, 1)],  # Job4
]

# Machineの数
machines_count = 1 + max(task[0] for job in jobs_data for task in job)
all_machines = range(machines_count)

# Taskの合計時間
horizon = sum(task[1] for job in jobs_data for task in job)

### モデルの作成

In [31]:
# モデルの作成
model = cp_model.CpModel()

### 変数の定義

In [32]:
# Taskの種類
task_type = collections.namedtuple('task_type', 'start end interval')

# 
assigned_task_type = collections.namedtuple('assigned_task_type',
                                            'start job index duration')

# Creates job intervals and add to the corresponding machine lists.
all_tasks = {}
machine_to_intervals = collections.defaultdict(list)

# 開始、終了、処理時間変数を作成
for job_id, job in enumerate(jobs_data):
    for task_id, task in enumerate(job):
        machine = task[0]
        duration = task[1]
        suffix = '_%i_%i' % (job_id, task_id)
        start_var = model.NewIntVar(0, horizon, 'start' + suffix)
        end_var = model.NewIntVar(0, horizon, 'end' + suffix)
        interval_var = model.NewIntervalVar(start_var, duration, end_var,
                                            'interval' + suffix)
        all_tasks[job_id, task_id] = task_type(start=start_var,
                                               end=end_var,
                                               interval=interval_var)
        machine_to_intervals[machine].append(interval_var)

### 制約の定義

In [33]:
# 重複無の制約を追加
for machine in all_machines:
    model.AddNoOverlap(machine_to_intervals[machine])

# 実行順序の制約を追加
for job_id, job in enumerate(jobs_data):
    for task_id in range(len(job) - 1):
        model.Add(all_tasks[job_id, task_id +
                            1].start >= all_tasks[job_id, task_id].end)

### 目的関数の定義
makespanを最小化する関数として定義する

In [34]:
# 目的関数
obj_var = model.NewIntVar(0, horizon, 'makespan')
model.AddMaxEquality(obj_var, [
    all_tasks[job_id, len(job) - 1].end
    for job_id, job in enumerate(jobs_data)
])
model.Minimize(obj_var)

### ソルバーの呼び出し

In [35]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

### 結果表示

In [36]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('Solution:')
    # Create one list of assigned tasks per machine.
    assigned_jobs = collections.defaultdict(list)
    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            machine = task[0]
            assigned_jobs[machine].append(
                assigned_task_type(start=solver.Value(
                    all_tasks[job_id, task_id].start),
                                   job=job_id,
                                   index=task_id,
                                   duration=task[1]))

    # Create per machine output lines.
    output = ''
    for machine in all_machines:
        # Sort by starting time.
        assigned_jobs[machine].sort()
        sol_line_tasks = 'Machine ' + str(machine) + ': '
        sol_line = '           '

        for assigned_task in assigned_jobs[machine]:
            name = 'job_%i_task_%i' % (assigned_task.job,
                                       assigned_task.index)
            # Add spaces to output to align columns.
            sol_line_tasks += '%-15s' % name

            start = assigned_task.start
            duration = assigned_task.duration
            sol_tmp = '[%i,%i]' % (start, start + duration)
            # Add spaces to output to align columns.
            sol_line += '%-15s' % sol_tmp

        sol_line += '\n'
        sol_line_tasks += '\n'
        output += sol_line_tasks
        output += sol_line

    # Finally print the solution found.
    print(f'Optimal Schedule Length: {solver.ObjectiveValue()}')
    print(output)
else:
    print('No solution found.')

Solution:
Optimal Schedule Length: 57.0
Machine 0: job_1_task_0   job_3_task_1   job_5_task_0   job_10_task_0  job_14_task_0  job_12_task_1  job_19_task_0  job_23_task_0  job_21_task_1  job_0_task_0   job_9_task_0   job_18_task_0  
           [0,2]          [2,3]          [3,5]          [5,7]          [7,9]          [9,10]         [10,12]        [12,14]        [14,15]        [15,18]        [18,21]        [21,24]        
Machine 1: job_2_task_0   job_4_task_2   job_1_task_2   job_8_task_2   job_13_task_2  job_5_task_2   job_17_task_2  job_22_task_2  job_10_task_2  job_26_task_2  job_0_task_1   job_9_task_1   job_18_task_1  job_11_task_0  job_14_task_2  job_19_task_2  job_20_task_0  job_23_task_2  
           [0,4]          [4,5]          [5,9]          [9,10]         [10,11]        [11,15]        [15,16]        [16,17]        [17,21]        [21,22]        [22,24]        [24,26]        [26,28]        [28,32]        [32,36]        [36,40]        [40,44]        [44,48]        
Machine 2: j